In [16]:
import os
import glob
import numpy as np

##################
# result_dir = "/home/galaxy/project/alleleSpecific_analysis/data/shFUS/peak/rm_m6Am/"
result_dir = "/home/galaxy/project/alleleSpecific_analysis/data/mouse_ASm6A/peak/rm_m6Am/"
os.system("mkdir -p %s" % result_dir)
os.chdir(result_dir)
##################

def count_reads(condition):
    ####
#     data_dir = "/home/galaxy/project/alleleSpecific_analysis/data/m6Am_inference/
    data_dir = "/home/galaxy/project/alleleSpecific_analysis/data/m6Am_inference/mm10/"
    left_region = "%s/left_region_100bp.bed" % data_dir
    right_region = "%s/right_region_100bp.bed" % data_dir
    ## different condition (run wt and kd seperately)
    bam_dir = "/home/galaxy/project/alleleSpecific_analysis/data/mouse_ASm6A/bam/%s/" % condition
    bam_list = glob.glob("%s/*.bam"%bam_dir)
    count_sh = os.path.join(result_dir, "%s_read_count.sh"%condition)
    ####
    with open(count_sh, 'w') as fw:
        bam_line = " ".join(["-bams %s"% os.path.abspath(bam) for bam in bam_list])
        left_res = os.path.join(result_dir, "%s_left_100bp_count.txt"%condition)
        fw.write("bedtools multicov %s -bed %s > %s\n" % (bam_line, left_region, left_res))
        right_res = os.path.join(result_dir, "%s_right_100bp_count.txt"%condition)
        fw.write("bedtools multicov %s -bed %s > %s\n" % (bam_line, right_region, right_res))

def generate_dict():
    # bed = "/home/galaxy/project/alleleSpecific_analysis/data/m6Am_inference/contain_a_TSS.bed"
    bed = "/home/galaxy/project/alleleSpecific_analysis/data/m6Am_inference/mm10/contain_a_TSS.bed"
    tss_dict = {}
    with open(bed, 'r') as f:
        contents = f.readlines()
        for i in range(len(contents)):
            tss_dict[i] = contents[i].strip()
    return tss_dict

def select_tss(in_file):
    row_nums = []
    with open(in_file, 'r') as f:
#         f.readline()
        lines = f.readlines()
        for i in range(len(lines)):
            input_count, ip_count = int(lines[i].split("\t")[6].strip()), int(lines[i].split("\t")[7].strip())
            if (input_count >= 5) and (ip_count >=5):
                row_nums.append(i)
            # num_list = [int(x) for x in lines[i].strip().split(",")[6:]]
#             edge_index = int(len(num_list)/2)
#             num_ip = len([x for x in num_list[:edge_index] if x >= 5])
#             num_input = len([x for x in num_list[edge_index:] if x >= 5])
#             if (num_ip >= np.ceil(edge_index/2)) and (num_input >= np.ceil(edge_index/2)):
#                 row_nums.append(i)
    return row_nums

# count_reads("WT")
# count_reads("KD")
# count_read
condition = "KD"  ############################################
tss_dict = generate_dict()
left_count, right_count = "%s_left_100bp_count.txt"%condition, "%s_right_100bp_count.txt"%condition
left_nums, right_nums = select_tss(left_count), select_tss(right_count)
total_nums = list(set(left_nums + right_nums))
with open("%s_selected_TSS.bed"%condition, 'w') as fw:
    for x in total_nums:
        fw.write(tss_dict[x] + "\n")
            
# ### overlap with m6a peak
tss = "%s_selected_TSS.bed"%condition
# m6a = "/home/galaxy/project/alleleSpecific_analysis/data/shFUS/peak/macs2_Fus2_normalized.bed" ###############
m6a = "/home/galaxy/project/alleleSpecific_analysis/data/mouse_ASm6A/peak/macs2_KD_normalized.bed" #############
m6am = "%s_m6Am.bed"%condition
os.system("bedtools intersect -a %s -b %s -wa > %s" % (m6a, tss, m6am))
    
# ### select m6a from raw m6a peak
real_m6a = "%s_m6A.bed"%condition
! grep -v -f $m6am $m6a > $real_m6a